In [2]:
import sklearn
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import cv2
from torchvision.transforms import ToTensor
import numpy as np
from tqdm import tqdm
import torchvision.transforms as transforms
from torchvision import datasets


In [ ]:
3, 224, 224

h = (h - k + 2*paddinf)/stride + 1

h = (224 - 3 + 2)/1 + 1 = 223 + 1 = 224
w = (224 - 3 + 2)/1 + 1 = 223 + 1 = 224

h = 112
w = 112

56 * 56

In [3]:
class EyeModel(nn.Module):
    
    def __init__(self, input_shape: int, hidden_units : int, output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),     
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                     out_channels=hidden_units,
                     kernel_size=3,
                     stride=1,
                     padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                     out_channels=hidden_units,
                     kernel_size=3,
                     stride=1,
                     padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units *56*56, out_features=output_shape)
        )
        
    def forward(self, x):
        x = self.conv_block_1(x)
#         print(x.shape)
        x = self.conv_block_2(x)
#         print(x.shape)
        x = self.classifier(x)
        return x

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# TRAINING FUNCTION
def train_step(model: torch.nn.Module,
              data_loader : torch.utils.data.dataloader,
              loss_fn : torch.nn.Module,
              optimizer : torch.optim.Optimizer,
              accuracy_fn,
              device : torch.device = device):
    train_loss, train_acc = 0, 0
    
    model.train()
    
    for batch, (x, y) in enumerate(data_loader):
        
        x, y = x.to(device), y.to(device)
        
        y_pred = model(x)
        
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        y_pred = torch.argmax(y_pred, dim=1)
        train_acc += (y_pred==y).sum().item()/len(y_pred)
        
        
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    
    train_loss/= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train Loss : {train_loss} | Train Acc : {train_acc}")
    return train_loss, train_acc
    
    

In [6]:
def test_step(model : torch.nn.Module,
             data_loader : torch.utils.data.dataloader,
             loss_fn : torch.nn.Module,
             accuracy_fn,
             device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval()
    
    with torch.inference_mode():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            test_pred = model(x)
            
            tes_loss = loss_fn(test_pred, y)
            test_loss += tes_loss.item()
            
            tes_acc = torch.argmax(torch.softmax(test_pred, dim=1), dim=1)
            test_acc += (tes_acc==y).sum().item()/len(tes_acc)
            
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test Loss : {test_loss} | Test Acc : {test_acc}")
        return test_loss, test_acc
        

In [7]:
import requests

from pathlib import Path


if Path("helper_functions.py").is_file():
    print("it already existst")
else:
    print("Downloading")
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open("helper_functions.py", "wb") as f:
        f.write(request.content)


it already existst


In [8]:
model = EyeModel(input_shape=3, hidden_units=10, output_shape=5).to(device)

In [9]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                           lr=0.1)

In [10]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = self.data.iloc[index, 0]
        img_path = train_images_folder + "/" + img_path + ".jpg"
#         image = Image.open(img_path).convert('RGB')
        image = cv2.imread(img_path)
        image = torch.tensor(image, dtype=torch.float32) / 255.0
        image = image.permute(2, 0, 1)
        
        label = self.data.iloc[index, 1]
        
#         print("Image path:", img_path)
#         print("Label:", label)

        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize the images to a consistent size
    transforms.RandomEqualize(),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder("C:\\Kavhi's Folder\\College\\Project\\DATA\\archive\\colored_images\\colored_images", transform=transform)
test_dataset = datasets.ImageFolder("C:\\Kavhi's Folder\\College\\archive\\colored_images\\Test_Data", transform=transform)


train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [14]:
image_batch, label_batch = next(iter(train_dataloader))
image_batch.shape, label_batch.shape

(torch.Size([4, 3, 224, 224]), torch.Size([4]))

In [16]:
import random
random.seed(42)
random.randint(1, 10)

2

In [17]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as timer
train_start_time = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch : {epoch}\n----------")
    train_step(model=model,
              data_loader=train_dataloader,
              loss_fn=loss_fn,
              optimizer=optimizer,
              accuracy_fn=accuracy_fn)
    test_step(model=model,
              data_loader=test_dataloader,
              loss_fn=loss_fn,
              accuracy_fn=accuracy_fn
              )
    
train_end_time = timer()
total_time = train_end_time - train_start_time

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Epoch : 0
----------
Train Loss : 0.873450767984425 | Train Acc : 0.7347130494192666


 33%|███████████████████████████▋                                                       | 1/3 [01:55<03:51, 115.79s/it]

Test Loss : 1.227682294550857 | Test Acc : 0.6125461254612546
Epoch : 1
----------
Train Loss : 0.8701508084748579 | Train Acc : 0.7347699840583011


 67%|████████████████████████████████████████████████████████                            | 2/3 [03:10<01:31, 91.50s/it]

Test Loss : 1.2067939704524635 | Test Acc : 0.6123154981549815
Epoch : 2
----------
Train Loss : 0.8701039269138946 | Train Acc : 0.7347699840583011


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:24<00:00, 88.13s/it]

Test Loss : 1.1874061904820128 | Test Acc : 0.6123923739237392


In [22]:
from torchinfo import summary

In [23]:
summary(model, input_size=[1, 3, 224, 224])

Layer (type:depth-idx)                   Output Shape              Param #
EyeModel                                 [1, 5]                    --
├─Sequential: 1-1                        [1, 10, 112, 112]         --
│    └─Conv2d: 2-1                       [1, 10, 224, 224]         280
│    └─ReLU: 2-2                         [1, 10, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 10, 224, 224]         910
│    └─ReLU: 2-4                         [1, 10, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 10, 112, 112]         --
├─Sequential: 1-2                        [1, 10, 56, 56]           --
│    └─Conv2d: 2-6                       [1, 10, 112, 112]         910
│    └─ReLU: 2-7                         [1, 10, 112, 112]         --
│    └─Conv2d: 2-8                       [1, 10, 112, 112]         910
│    └─ReLU: 2-9                         [1, 10, 112, 112]         --
│    └─MaxPool2d: 2-10                   [1, 10, 56, 56]           --
├─Sequentia

In [23]:
from tqdm import tqdm

def train(model: torch.nn.Module,
         train_dataloader: torch.utils.data.dataloader,
         test_dataloader: torch.utils.data.dataloader,
         optimizer: torch.optim.Optimizer,
         loss_fn: torch.nn.Module,
         epochs: int = 5,
         device = "CPU"):
    
    results = {"train_loss":[],
              "train_acc":[],
              "test_loss":[],
              "test_acc":[]}
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          data_loader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          accuracy_fn=None,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
                                       data_loader=test_dataloader,
                                       loss_fn=loss_fn,
                                       accuracy_fn=None,
                                       device=device)
        
        print(f"Epoch : {epoch} | Train Loss : {train_loss:.4f} | Train Acc : {train_acc:.4f} | Test Loss : {test_loss:.4f} | Test Acc : {test_acc:.4f}")
        
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)
        results["train_acc"].append(train_acc)
        results["test_acc"].append(test_acc)
        
    return results